# Introduction to SciPy
Tutorial at EuroSciPy 2019, Bilbao
## 1. Statistical analysis – Gyroscope data taken in a TGV

In [ ]:
import numpy as np
from scipy import optimize, stats
%matplotlib notebook
import matplotlib.pyplot as plt

### Importing the data

Our gyroscope data are stored in a compressed file `data/TGV_data.csv.bz2`. Each row of the uncompressed file contains entries separated by commas and the first row contains labels explaining the content of the respective column.

In [ ]:
data = np.genfromtxt('data/TGV_data.csv.bz2', delimiter=',', names=True)

There are five columns identified by names:

In [ ]:
data.dtype.names

In [ ]:
time = data['Time_s']
omega_x = data['Gyroscope_x_rads']

Let us first get an idea of the data.

In [ ]:
_ = plt.plot(time, omega_x)

### Statistical analysis

We use the data for $\omega_x$ to demonstrate some aspects of statistical analysis with SciPy. Let us first take a look at a histogram of the data.

In [ ]:
n, bins = np.histogram(omega_x, bins=100, density=True)

Because we have set `density=True`, the data can be considered as a normalized probability distribution.

In [ ]:
np.sum(n)*(bins[1]-bins[0])

<div style="color: DarkBlue;background-color: LightYellow">
    <i>Exercise:</i> Check the values of n. What happens if you set density=False?
</div>

The array `bins` contains the edges of the bins. To plot the histogram, we need their centers.

In [ ]:
bincenters = 0.5*(bins[1:]+bins[:-1])

In [ ]:
_ = plt.plot(bincenters, n)

With SciPy we can easily determine some statistical characteristics from the original data:

In [ ]:
description = stats.describe(omega_x, ddof=False)
description

* mean $\langle x\rangle$


* variance $\langle (x-\langle x\rangle)^2\rangle$


* skewness $\dfrac{\langle(x-\langle x\rangle)^3\rangle}{\langle(x-\langle x\rangle)^2\rangle^{3/2}}$


* kurtosis $\dfrac{\langle(x-\langle x\rangle)^4\rangle}{\langle(x-\langle x\rangle)^2\rangle^2}-3$

<div style="color: DarkBlue;background-color: LightYellow">
    <i>Exercise:</i> Check the result for the kurtosis as there are two different definitions with 3 subtracted or not.
</div>

We can compare our histogram with a Gaussian for the mean and variance just obtained.

Probability density function of the normal distribution
$$p(x) = \frac{1}{\sqrt{2\pi}}\exp\left(-\frac{x^2}{2}\right)$$

For general mean and variance:
$$x = \frac{y-\mathrm{loc}}{\mathrm{scale}}$$

In [ ]:
loc = description.mean
scale = np.sqrt(description.variance)
loc, scale

In [ ]:
x = np.linspace(-0.1, 0.1, 200)
_ = plt.plot(x, stats.norm.pdf(x, loc, scale))

A maximum likelihood fit with a Gaussian will yield the same result.

In [ ]:
stats.norm.fit(omega_x)

How can we fit the histogram to a Gaussian? Use nonlinear least-squares curve fitting.

In [ ]:
def gaussian(x, loc, scale):
    return stats.norm.pdf(x, loc, scale)

In [ ]:
popt, pcov = optimize.curve_fit(gaussian, bincenters, n)
popt

In [ ]:
_ = plt.plot(x, gaussian(x, *popt))

How likely is it that our data follow a normal distribution?

In [ ]:
stats.normaltest(omega_x)

<div style="color: DarkBlue;background-color: LightYellow">
    <i>Exercise:</i> Differ the results for the <i>y</i>-component of the angular velocity from those for the <i>x</i>-component?
</div>

### Generation of normally distributed data

In [ ]:
normdata = stats.norm.rvs(1, 0.2, 5000)
_ = plt.plot(normdata)

In [ ]:
n, bins = np.histogram(normdata, bins=100, density=True)
bincenters = 0.5*(bins[1:]+bins[:-1])
plt.plot(bincenters, n)

In [ ]:
x = np.linspace(0, 1.6, 200)
_ = plt.plot(x, stats.norm.pdf(x, *stats.norm.fit(normdata)))

In [ ]:
popt, pcov = optimize.curve_fit(gaussian, bincenters, n)
popt

In [ ]:
_ = plt.plot(x, gaussian(x, *popt))

In [ ]:
stats.normaltest(normdata)

<div style="color: DarkBlue;background-color: LightYellow">
    <i>Exercise:</i> Do these results depend on the number of samples and the realization of the random variates?
</div>

### Distribution with skewness

In [ ]:
for a in range(-4, 5, 2):
    x = np.linspace(stats.skewnorm.ppf(0.001, a),
                    stats.skewnorm.ppf(0.999, a), 100)
    plt.plot(x, stats.skewnorm.pdf(x, a))